In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import os
import math
import data_loader
import ResNet as models
#from torch.utils import model_zoo
from config.miccai_settings import *
from utils.data_preprocess import *
import dataset_loader as dl
from torch.utils.tensorboard import SummaryWriter
import sys
from pathlib import Path

In [5]:
def train(epoch, model, optimizer):

    optimizer.param_groups[0]['lr'] = lr[0] / math.pow((1 + 10 * (epoch - 1) / epochs), 0.75)
    optimizer.param_groups[1]['lr'] = lr[1] / math.pow((1 + 10 * (epoch - 1) / epochs), 0.75)

    model.train()

    iter_source_train = iter(source_train_loader)
    num_iter_train = len_source_train_loader
    for i in range(1, num_iter_train):
        data_source_train, label_source_train = iter_source_train.next()
        if cuda:
            data_source_train, label_source_train = data_source_train.cuda(), label_source_train.cuda()
        data_source_train, label_source_train = Variable(data_source_train), Variable(label_source_train)

        optimizer.zero_grad()
        label_source_train_pred = model(data_source_train)
        loss = F.cross_entropy(F.log_softmax( label_source_train_pred, dim=1), label_source_train.type(torch.long))
        loss.backward()
        optimizer.step()

        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tsoft_Loss: {:.6f}'.format(
                epoch, i * len(data_source_train), len_source_train_dataset,
                100. * i / len_source_train_loader, loss.data, loss.data))


In [6]:
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#torch.cuda.set_device(1)
writer = SummaryWriter('runs')

# Training settings
st = Settings()
options = st.get_options()

batch_size = options['batch_size']
epochs = options['max_epochs']
lr = [0.001, 0.01]
momentum = 0.9
no_cuda =False
seed = options['seed']
log_interval = 10
l2_decay = 5e-4
source_path = options['train_folder']
source_name = ''
cuda = not no_cuda and torch.cuda.is_available()

# resize images in path
#resize_images(options)

# generate csv file
df = miccai_generate_csv(options)

# split data to train, validate folds
miccai_split_folds(options['train_csv_path'], options['seed'], options['k_fold'])

# list scan
fold = 0
# fold train data
df = pd.read_csv(options['train_csv_path'])
# select training scans
train_files = df.loc[df['fold'] != fold, ['center_id','patient']].values
valid_files = df.loc[df['fold'] == fold, ['center_id', 'patient']].values
train_scan_list = [f[0]+'_'+f[1] for f in train_files]
valid_scan_list = [f[0]+'_'+f[1] for f in valid_files]

train_scan_list.sort()
valid_scan_list.sort()

train_x_data = {f: {m: os.path.join(options['train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in train_scan_list}
train_y_data = {f: os.path.join(options['train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in train_scan_list}

valid_x_data = {f: {m: os.path.join(options['train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in valid_scan_list}
valid_y_data = {f: os.path.join(options['train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in valid_scan_list}


torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

source_train_loader = dl.load_training(options, train_x_data, train_y_data)
source_valid_loader = dl.load_training(options, valid_x_data, valid_y_data)

#source_test_loader = data_loader.load_testing('', source_path, batch_size, kwargs)

len_source_train_dataset = len(source_train_loader.dataset)
len_source_valid_dataset = len(source_valid_loader.dataset)
len_source_train_loader = len(source_train_loader)
len_source_valid_loader = len(source_valid_loader)

model = models.DANNet_source(num_classes=2)
writer.add_graph(model, torch.rand(size=(128,2,16,16,16)))
writer.flush()
writer.close()
#sys.exit()
correct = 0
print(model)
if cuda:
    model.cuda()

optimizer = torch.optim.SGD([
    {'params': model.sharedNet.parameters()},
    {'params': model.cls_fc.parameters(), 'lr': lr[1]},
    ], lr=lr[0], momentum=momentum, weight_decay=l2_decay)
path= os.path.join(options['weight_paths'],options['experiment'])
Path(path).mkdir(parents=True, exist_ok=True)

> CNN: loading training data for first model


/home/mostafa/Marwa/Code/deep-transfer-learning/UDA/pytorch0.3/DAN/utils/data_load.py:366: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  patches = [new_image[idx] for idx in slices]


shape (66516, 16, 16, 16)
center: [33258, 8, 8, 8]
> CNN: train_x  (66516, 2, 16, 16, 16) train_y  (66516,)
> CNN: loading training data for first model
shape (176966, 16, 16, 16)
center: [88483, 8, 8, 8]
> CNN: train_x  (176966, 2, 16, 16, 16) train_y  (176966,)
DANNet_source(
  (sharedNet): ResNet(
    (conv1): Conv3d(2, 64, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1

In [7]:
epoch=1
train(epoch, model, optimizer)

Train Epoch: 1 [1280/66516 (2%)]	Loss: 1.080118	soft_Loss: 1.080118
Train Epoch: 1 [2560/66516 (4%)]	Loss: 0.438354	soft_Loss: 0.438354
Train Epoch: 1 [3840/66516 (6%)]	Loss: 0.818090	soft_Loss: 0.818090
Train Epoch: 1 [5120/66516 (8%)]	Loss: 0.722782	soft_Loss: 0.722782
Train Epoch: 1 [6400/66516 (10%)]	Loss: 0.302753	soft_Loss: 0.302753
Train Epoch: 1 [7680/66516 (12%)]	Loss: 0.257738	soft_Loss: 0.257738
Train Epoch: 1 [8960/66516 (13%)]	Loss: 0.158648	soft_Loss: 0.158648
Train Epoch: 1 [10240/66516 (15%)]	Loss: 0.291947	soft_Loss: 0.291947
Train Epoch: 1 [11520/66516 (17%)]	Loss: 0.920967	soft_Loss: 0.920967
Train Epoch: 1 [12800/66516 (19%)]	Loss: 0.299721	soft_Loss: 0.299721
Train Epoch: 1 [14080/66516 (21%)]	Loss: 0.149805	soft_Loss: 0.149805
Train Epoch: 1 [15360/66516 (23%)]	Loss: 0.175400	soft_Loss: 0.175400
Train Epoch: 1 [16640/66516 (25%)]	Loss: 0.241831	soft_Loss: 0.241831
Train Epoch: 1 [17920/66516 (27%)]	Loss: 0.168030	soft_Loss: 0.168030
Train Epoch: 1 [19200/66516 (29

In [18]:
with torch.no_grad():
    model.eval()
    test_loss = 0
    correct = 0
    iter_source_valid = iter(source_valid_loader)
    num_iter_valid = len_source_valid_loader
    for i in range(1, num_iter_valid):
        data_source_valid, label_source_valid = iter_source_valid.next()
        if cuda:
            data_source_valid, label_source_valid = data_source_valid.cuda(), label_source_valid.cuda()
        data_source_valid, label_source_valid = Variable(data_source_valid), Variable(label_source_valid)
        #print(data_source_valid.requires_grad)
       
        s_output = model(data_source_valid)
        test_loss += F.cross_entropy(F.log_softmax(s_output, dim = 1), label_source_valid.type(torch.long)) # sum up batch loss
        pred = s_output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(label_source_valid.view_as(pred)).cpu().sum()
        if i == 21:
            pass
        print(i)

    test_loss /= len_source_valid_dataset
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        source_name, test_loss, correct, len_source_valid_dataset,
        100. * correct / len_source_valid_dataset))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
    
    for epoch in range(1, epochs + 1):
        train(epoch, model, optimizer)
        #torch.cuda.synchronize()
        t_correct = validate(model)
        
        FILE = os.path.join(path,str(epoch)+'_model.pth')
        torch.save(model, FILE)

        if t_correct > correct:
            correct = t_correct
        #correct = correct.item()
        print(correct)
        print('source: {} to target: {} max correct: {} max accuracy{: .2f}%\n'.format(
              source_name, '', correct, 100. * correct / len_source_valid_dataset))

In [10]:
def validate(model):
    model.eval()
    test_loss = 0
    correct = 0
    iter_source_valid = iter(source_valid_loader)
    num_iter_valid = len_source_valid_loader
    for i in range(1, num_iter_valid):
        data_source_valid, label_source_valid = iter_source_valid.next()
        if cuda:
            data_source_valid, label_source_valid = data_source_valid.cuda(), label_source_valid.cuda()
        data_source_valid, label_source_valid = Variable(data_source_valid, volatile=True), Variable(label_source_valid, volatile=True)
        s_output = model(data_source_valid)
        test_loss += F.cross_entropy(F.log_softmax(s_output, dim = 1), label_source_valid.type(torch.long)) # sum up batch loss
        pred = s_output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(label_source_valid.view_as(pred)).cpu().sum()
        if i == 21:
            pass
        print(i)

    test_loss /= len_source_valid_dataset
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        source_name, test_loss, correct, len_source_valid_dataset,
        100. * correct / len_source_valid_dataset))
    return correct
